In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import csv
import numpy as np
import sklearn
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data cleaning

In [ ]:
years = [year for year in range(2021, 2009, -1)]

In [ ]:
import re

def process_excel_file(year, excel_path):
    df = pd.read_excel(excel_path, sheet_name=1, header=None)


    df = df.transpose().reset_index()
    df.columns = df.iloc[0]
    # Drop the first row
    df = df.iloc[1:]

    df.columns = ['Owner_occupied_' + col if 10>= i > 4 else col for i, col in enumerate(df.columns)]
    df.columns = ['Renter_occupied_' + col if 17>= i > 11 else col for i, col in enumerate(df.columns)]
    df = df.rename(columns={'Owner occupied:' : 'Owner_occupied', 'Renter occupied:' : 'Renter_occupied'})

    new_column_name = 'census_tract'  # Replace with the new name
    df.columns.values[1] = new_column_name
    df = df.dropna(subset=['census_tract'])
    #df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if isinstance(x, str) else x)
    df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9]', '', x) if isinstance(x, str) else x)
    df = df.reset_index()
    df = df.drop(columns = ['Label', 'index', 0], axis=1)
    df['year'] = year




    return df

In [ ]:
for year in years:
  excel_file_path = f'/content/drive/MyDrive/DS project/Dorchester_homeownership/ACSDT5Y{year}.xlsx'
  variable_name = f'Dorchester_homeownership_{year}'

        # Call the function and assign the result to the variable
  globals()[variable_name] = process_excel_file(year, excel_file_path)

In [ ]:
Dorchester_homeownership_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   census_tract                                 29 non-null     object
 1   Total population in occupied housing units:  29 non-null     object
 2   Owner_occupied                               29 non-null     object
 3   Owner_occupied_Moved in 2019 or later        29 non-null     object
 4   Owner_occupied_Moved in 2015 to 2018         29 non-null     object
 5   Owner_occupied_Moved in 2010 to 2014         29 non-null     object
 6   Owner_occupied_Moved in 2000 to 2009         29 non-null     object
 7   Owner_occupied_Moved in 1990 to 1999         29 non-null     object
 8   Owner_occupied_Moved in 1989 or earlier      29 non-null     object
 9   Renter_occupied                              29 non-null     object
 10  Renter_occupied_

In [ ]:
for year in years:
  df = globals()[f'Dorchester_homeownership_{year}']
  for column in df.columns[: -1]:
    df[column] = df[column].str.replace('[^0-9-]', '', regex=True).astype(int)

In [ ]:
for df in [Dorchester_homeownership_2021, Dorchester_homeownership_2020, Dorchester_homeownership_2019, Dorchester_homeownership_2018]:

  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 7:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 14:16].sum(axis=1)

In [ ]:
for df in [Dorchester_homeownership_2017, Dorchester_homeownership_2016, Dorchester_homeownership_2015]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 6:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 13:16].sum(axis=1)

In [ ]:
for df in [Dorchester_homeownership_2014, Dorchester_homeownership_2013, Dorchester_homeownership_2012, Dorchester_homeownership_2011, Dorchester_homeownership_2010]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 5:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 12:16].sum(axis=1)

In [ ]:
Dorchester_homeownership_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   census_tract                                 29 non-null     int64
 1   Total population in occupied housing units:  29 non-null     int64
 2   Owner_occupied                               29 non-null     int64
 3   Owner_occupied_Moved in 2010 or later        29 non-null     int64
 4   Owner_occupied_Moved in 2000 to 2009         29 non-null     int64
 5   Owner_occupied_Moved in 1990 to 1999         29 non-null     int64
 6   Owner_occupied_Moved in 1980 to 1989         29 non-null     int64
 7   Owner_occupied_Moved in 1970 to 1979         29 non-null     int64
 8   Owner_occupied_Moved in 1969 or earlier      29 non-null     int64
 9   Renter_occupied                              29 non-null     int64
 10  Renter_occupied_Moved in 201

In [ ]:
#list_of_indices
# Create a list of DataFrames
concatenated_df = [globals()[f'Dorchester_homeownership_{year}'] for year in years]

# Initialize an empty DataFrame to store the concatenated result
result_df2 = pd.DataFrame()

# Concatenate the DataFrames in the list
for df in concatenated_df:
    result_df2 = pd.concat([result_df2, df], ignore_index=True)

In [ ]:
result_df2.columns

Index(['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 2019 or later',
       'Owner_occupied_Moved in 2015 to 2018',
       'Owner_occupied_Moved in 2010 to 2014',
       'Owner_occupied_Moved in 2000 to 2009',
       'Owner_occupied_Moved in 1990 to 1999',
       'Owner_occupied_Moved in 1989 or earlier', 'Renter_occupied',
       'Renter_occupied_Moved in 2019 or later',
       'Renter_occupied_Moved in 2015 to 2018',
       'Renter_occupied_Moved in 2010 to 2014',
       'Renter_occupied_Moved in 2000 to 2009',
       'Renter_occupied_Moved in 1990 to 1999',
       'Renter_occupied_Moved in 1989 or earlier', 'year',
       'Owner_occupied_Moved in 1999 or earlier',
       'Renter_occupied_Moved in 1999 or earlier',
       'Owner_occupied_Moved in 2017 or later',
       'Owner_occupied_Moved in 2015 to 2016',
       'Renter_occupied_Moved in 2017 or later',
       'Renter_occupied_Moved in 2015 to 2016',
       'Owner_

In [ ]:
result_df3 = result_df2[['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 1999 or earlier',
      'Renter_occupied', 'Renter_occupied_Moved in 1999 or earlier',
       'year']]

In [ ]:
result_df3.sort_values(by=['census_tract', 'year'], inplace=True)

result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
result_df3['percent_renters'] = result_df3['Renter_occupied']/result_df3['Total population in occupied housing units:']

result_df3['percent_old_owners'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Owner_occupied']
result_df3['percent_old_renters'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied']

result_df3['renters_owners_ratio'] = result_df3['Owner_occupied']/result_df3['Renter_occupied']
result_df3['old_renters_owners_ratio'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied_Moved in 1999 or earlier']

# Calculate the percentage change within each 'census_tract' group
#result_df3['percentage_change'] = result_df3.groupby('census_tract')['Owner_occupied_Moved in 1999 or earlier'].pct_change()

<ipython-input-19-236f53da9bbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3.sort_values(by=['census_tract', 'year'], inplace=True)
<ipython-input-19-236f53da9bbd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
<ipython-input-19-236f53da9bbd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
# Adding back the decimal to census tract..
def custom_replace(value):
    if value > 10000:
        return value/100
   # else: return int(value)

    return value


result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)

<ipython-input-21-424f84e325ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].astype('float64')


In [ ]:
# Adding back the decimal to census tract..
def custom_replace(value):
    if value > 10000:
        return value/100
   # else: return int(value)

    return value


result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)

<ipython-input-23-d5d10f3af895>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)


In [ ]:
result_df3.reset_index(drop=True)

,census_tract,Total population in occupied housing units:,Owner_occupied,Owner_occupied_Moved in 1999 or earlier,Renter_occupied,Renter_occupied_Moved in 1999 or earlier,year,percent_owners,percent_renters,percent_old_owners,percent_old_renters,renters_owners_ratio,old_renters_owners_ratio
0,901.0,4658,1351,699,3307,916,2010,0.290039,0.709961,0.517395,0.211370,0.408527,0.763100
1,901.0,4941,1580,804,3361,647,2011,0.319773,0.680227,0.508861,0.239215,0.470098,1.242658
2,901.0,4941,1317,712,3624,482,2012,0.266545,0.733455,0.540623,0.196468,0.363411,1.477178
3,901.0,4986,926,689,4060,400,2013,0.185720,0.814280,0.744060,0.169704,0.228079,1.722500
4,901.0,5108,913,677,4195,250,2014,0.178739,0.821261,0.741512,0.161383,0.217640,2.708000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,100603.0,2049,1210,321,839,11,2017,0.590532,0.409468,0.265289,0.382598,1.442193,29.181818
342,100603.0,1953,1094,298,859,0,2018,0.560164,0.439836,0.272395,0.346915,1.273574,inf
343,100603.0,1894,1136,217,758,18,2019,0.599789,0.400211,0.191021,0.286280,1.498681,12.055556
344,100603.0,2032,1344,476,688,9,2020,0.661417,0.338583,0.354167,0.691860,1.953488,52.888889


In [ ]:
result_df3.to_csv('/content/drive/MyDrive/DS project/Dorchester_homeownership/Dorchester_homeownership.csv')

Igonre the following......


In [ ]:
Dorchester_homeownership_2021_2020 = pd.concat([Dorchester_homeownership_2021, Dorchester_homeownership_2020])

Dorchester_homeownership_2019_2018 = pd.concat([Dorchester_homeownership_2019, Dorchester_homeownership_2018])

Dorchester_homeownership_2017_2016_2015 = pd.concat([Dorchester_homeownership_2017, Dorchester_homeownership_2016, Dorchester_homeownership_2015])

Dorchester_homeownership_2014_2013_2012 = pd.concat([Dorchester_homeownership_2014, Dorchester_homeownership_2013, Dorchester_homeownership_2012])

Dorchester_homeownership_2011_2010 = pd.concat([Dorchester_homeownership_2011, Dorchester_homeownership_2010])

In [ ]:
Dorchester_homeownership_2021_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 28
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   census_tract                                 58 non-null     object
 1   Total population in occupied housing units:  58 non-null     object
 2   Owner_occupied                               58 non-null     object
 3   Owner_occupied_Moved in 2019 or later        58 non-null     object
 4   Owner_occupied_Moved in 2015 to 2018         58 non-null     object
 5   Owner_occupied_Moved in 2010 to 2014         58 non-null     object
 6   Owner_occupied_Moved in 2000 to 2009         58 non-null     object
 7   Owner_occupied_Moved in 1990 to 1999         58 non-null     object
 8   Owner_occupied_Moved in 1989 or earlier      58 non-null     object
 9   Renter_occupied                              58 non-null     object
 10  Renter_occupied_

In [ ]:
#list_of_indices
# Create a list of DataFrames
concatenated_df = [globals()[f'Dorchester_homeownership_{year}'] for year in years]

# Initialize an empty DataFrame to store the concatenated result
result_df = pd.DataFrame()

# Concatenate the DataFrames in the list
for df in concatenated_df:
    result_df = pd.concat([result_df, df], ignore_index=True)

In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 37 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   census_tract                                 346 non-null    object
 1   Total population in occupied housing units:  346 non-null    object
 2   Owner_occupied                               346 non-null    object
 3   Owner_occupied_Moved in 2019 or later        58 non-null     object
 4   Owner_occupied_Moved in 2015 to 2018         58 non-null     object
 5   Owner_occupied_Moved in 2010 to 2014         201 non-null    object
 6   Owner_occupied_Moved in 2000 to 2009         288 non-null    object
 7   Owner_occupied_Moved in 1990 to 1999         346 non-null    object
 8   Owner_occupied_Moved in 1989 or earlier      114 non-null    object
 9   Renter_occupied                              346 non-null    object
 10  Renter_occupie

In [ ]:
result_df.to_csv('/content/drive/MyDrive/DS project/Dorchester_homeownership/sample_alltract.xlsx')

In [ ]:
for year in years:
  df = globals()[f'Dorchester_homeownership_{year}']
  for column in df.columns[: -1]:
    df[column] = df[column].str.replace('[^0-9-]', '', regex=True).astype(int)

In [ ]:
result

,census_tract,Total population in occupied housing units:,owner_occupied,Owner_occupied_Moved in 2005 or later,Owner_occupied_Moved in 2000 to 2004,Owner_occupied_Moved in 1990 to 1999,Owner_occupied_Moved in 1980 to 1989,Owner_occupied_Moved in 1970 to 1979,Owner_occupied_Moved in 1969 or earlier,renter_occupied,Renter_occupied_Moved in 2005 or later,Renter_occupied_Moved in 2000 to 2004,Renter_occupied_Moved in 1990 to 1999,Renter_occupied_Moved in 1980 to 1989,Renter_occupied_Moved in 1970 to 1979,Renter_occupied_Moved in 1969 or earlier,year
0,901,4941,1580,392,384,457,48,39,260,3361,1814,900,356,112,113,66,2011
1,902,2422,212,68,0,21,0,0,123,2210,1331,500,291,34,11,43,2011
2,903,2551,588,175,23,43,41,188,118,1963,1217,565,129,12,27,13,2011
3,907,3972,1058,245,235,176,212,42,148,2914,1877,644,252,89,28,24,2011
4,90901,3940,0,0,0,0,0,0,0,3940,2744,466,478,188,11,53,2011
5,91001,2668,1338,436,238,171,346,104,43,1330,737,387,146,0,17,43,2011
6,911,4628,2112,555,609,496,150,56,246,2516,1915,144,313,25,0,119,2011
7,912,2781,1258,151,159,518,216,65,149,1523,980,355,155,28,0,5,2011
8,913,1886,381,84,19,34,227,12,5,1505,825,543,94,0,43,0,2011
9,914,1884,718,235,62,137,130,49,105,1166,772,110,266,0,0,18,2011


In [ ]:
result.sort_values(by=['census_tract', 'year'], inplace=True)

# Calculate the percentage change within each 'census_tract' group
result['percentage_change'] = result.groupby('census_tract')['Owner_occupied_Moved in 2005 or later'].pct_change()

In [ ]:
 columns_percentage_cal = ['owner_occupied', 'Owner_occupied_Moved in 2005 or later',
       'Owner_occupied_Moved in 2000 to 2004',
       'Owner_occupied_Moved in 1990 to 1999',
       'Owner_occupied_Moved in 1980 to 1989',
       'Owner_occupied_Moved in 1970 to 1979',
       'Owner_occupied_Moved in 1969 or earlier', 'renter_occupied',
       'Renter_occupied_Moved in 2005 or later',
       'Renter_occupied_Moved in 2000 to 2004',
       'Renter_occupied_Moved in 1990 to 1999',
       'Renter_occupied_Moved in 1980 to 1989',
       'Renter_occupied_Moved in 1970 to 1979',
       'Renter_occupied_Moved in 1969 or earlier']



In [ ]:
five_dfs = [Roxbury_homeownership_2021_2020,

Roxbury_homeownership_2019_2018,

Roxbury_homeownership_2017_2016_2015,
Roxbury_homeownership_2014_2013_2012,

Roxbury_homeownership_2011_2010]

In [ ]:
for df in five_dfs:
  selected_columns_to_calculate = df.iloc[:, range(2,16)]
  for column in  selected_columns_to_calculate.columns:
    df[f'{column}_percentage_change'] = df.groupby('census_tract')[column].pct_change()


In [ ]:
selected_columns_to_calculate.columns

Index(['Owner_occupied', 'Owner_occupied_Moved in 2005 or later',
       'Owner_occupied_Moved in 2000 to 2004',
       'Owner_occupied_Moved in 1990 to 1999',
       'Owner_occupied_Moved in 1980 to 1989',
       'Owner_occupied_Moved in 1970 to 1979',
       'Owner_occupied_Moved in 1969 or earlier', 'Renter_occupied',
       'Renter_occupied_Moved in 2005 or later',
       'Renter_occupied_Moved in 2000 to 2004',
       'Renter_occupied_Moved in 1990 to 1999',
       'Renter_occupied_Moved in 1980 to 1989',
       'Renter_occupied_Moved in 1970 to 1979',
       'Renter_occupied_Moved in 1969 or earlier'],
      dtype='object')

In [ ]:
Roxbury_homeownership_2017_2016_2015.to_xlsx('/content/drive/MyDrive/DS project/Roxbury_homeownership/2017_2016_2015.xlsx')

In [ ]:
for column in  columns_percentage_cal:
    result[f'{column}_percentage_change'] = result.groupby('census_tract')[column].pct_change()


In [ ]:
Roxbury_homeownership_2021

,0,census_tract,Label,Total population in occupied housing units:,Owner_occupied_Owner occupied:,Owner_occupied_Moved in 2019 or later,Owner_occupied_Moved in 2015 to 2018,Owner_occupied_Moved in 2010 to 2014,Owner_occupied_Moved in 2000 to 2009,Owner_occupied_Moved in 1990 to 1999,Moved in 1989 or earlier,Renter_occupiedRenter occupied:,Renter_occupiedMoved in 2019 or later,Renter_occupiedMoved in 2015 to 2018,Renter_occupiedMoved in 2010 to 2014,Renter_occupiedMoved in 2000 to 2009,Renter_occupiedMoved in 1990 to 1999,Moved in 1989 or earlier
1,1,"Census Tract 901, Suffolk County, Massachusetts",Estimate,"5,185",771,0,32,159,89,180,311,"4,414",230,"1,531","1,452",951,101,149
2,2,NaN,Margin of Error,±803,±300,±19,±49,±120,±126,±151,±184,±734,±259,±565,±493,±381,±79,±157
3,3,"Census Tract 902, Suffolk County, Massachusetts",Estimate,"3,084",516,11,25,19,105,264,92,"2,568",210,678,480,"1,022",161,17
4,4,NaN,Margin of Error,±727,±259,±17,±48,±27,±128,±218,±67,±845,±225,±398,±268,±757,±106,±19
5,5,"Census Tract 903, Suffolk County, Massachusetts",Estimate,"3,206",404,43,53,34,38,150,86,"2,802",870,734,732,402,45,19
6,6,NaN,Margin of Error,±684,±227,±62,±57,±72,±64,±151,±69,±707,±393,±539,±423,±244,±48,±29
7,7,"Census Tract 907, Suffolk County, Massachusetts",Estimate,"4,059","1,648",0,257,344,670,172,205,"2,411",261,"1,155",504,292,21,178
8,8,NaN,Margin of Error,±512,±460,±13,±163,±219,±468,±166,±117,±441,±166,±379,±208,±185,±35,±150
9,9,"Census Tract 909.01, Suffolk County, Massachus...",Estimate,"3,042",0,0,0,0,0,0,0,"3,042",789,"1,414",452,327,40,20
10,10,NaN,Margin of Error,±446,±13,±13,±13,±13,±13,±13,±13,±446,±285,±353,±185,±176,±39,±22


In [ ]:
import re
df = pd.read_excel('/content/drive/MyDrive/DS project/Roxbury_homeownership/ACSDT5Y2010.xlsx', sheet_name=1)

df = df.transpose().reset_index()
df.columns = df.iloc[0]
# Drop the first row
df = df.iloc[1:]

df.columns = ['Owner_occupied_' + col if 9>= i >= 4 else col for i, col in enumerate(df.columns)]
df.columns = ['Renter_occupied' + col if 16>= i >= 11 else col for i, col in enumerate(df.columns)]

df = df[~df['Unnamed: 0'].str.contains('Unnamed')]

#df['Unnamed: 0'] = df['Unnamed: 0'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if isinstance(x, str) else x)
df['Unnamed: 0'] = df['Unnamed: 0'].apply(lambda x: re.sub(r'[^0-9]', '', x) if isinstance(x, str) else x)
df = df.reset_index()
df = df.drop(columns = ['Label', 'index'], axis=1)


In [ ]:
df


,Unnamed: 0,Total population in occupied housing units:,Owner occupied:,Owner_occupied_Moved in 2005 or later,Owner_occupied_Moved in 2000 to 2004,Owner_occupied_Moved in 1990 to 1999,Owner_occupied_Moved in 1980 to 1989,Owner_occupied_Moved in 1970 to 1979,Owner_occupied_Moved in 1969 or earlier,Renter occupied:,Renter_occupiedMoved in 2005 or later,Renter_occupiedMoved in 2000 to 2004,Renter_occupiedMoved in 1990 to 1999,Renter_occupiedMoved in 1980 to 1989,Renter_occupiedMoved in 1970 to 1979,Renter_occupiedMoved in 1969 or earlier
0,901,"4,658","1,351",341,311,441,56,39,163,"3,307","1,678",713,530,180,132,74
1,902,"2,075",214,80,20,9,0,0,105,"1,861",810,641,318,29,0,63
2,903,"2,525",591,151,34,167,56,90,93,"1,934","1,142",557,204,0,20,11
3,907,"4,157","1,279",252,441,210,216,31,129,"2,878","1,435","1,009",268,112,10,44
4,90901,"4,026",0,0,0,0,0,0,0,"4,026","2,439",904,479,204,0,0
5,91001,"2,479","1,217",300,268,206,205,113,125,"1,262",466,555,175,0,17,49
6,911,"4,344","2,097",368,453,695,333,51,197,"2,247","1,251",554,342,63,0,37
7,912,"2,714","1,189",78,69,534,302,98,108,"1,525",768,457,259,14,0,27
8,913,"1,741",271,46,17,42,151,15,0,"1,470",669,523,193,0,22,63
9,914,"2,008",716,176,75,202,193,0,70,"1,292",738,124,280,0,0,150


In [ ]:
df

,Unnamed: 0,Total population in occupied housing units:,Owner occupied:,Owner_occupied_Moved in 2005 or later,Owner_occupied_Moved in 2000 to 2004,Owner_occupied_Moved in 1990 to 1999,Owner_occupied_Moved in 1980 to 1989,Owner_occupied_Moved in 1970 to 1979,Owner_occupied_Moved in 1969 or earlier,Renter occupied:,Renter_occupiedMoved in 2005 or later,Renter_occupiedMoved in 2000 to 2004,Renter_occupiedMoved in 1990 to 1999,Renter_occupiedMoved in 1980 to 1989,Renter_occupiedMoved in 1970 to 1979,Renter_occupiedMoved in 1969 or earlier
0,901,"4,658","1,351",341,311,441,56,39,163,"3,307","1,678",713,530,180,132,74
1,902,"2,075",214,80,20,9,0,0,105,"1,861",810,641,318,29,0,63
2,903,"2,525",591,151,34,167,56,90,93,"1,934","1,142",557,204,0,20,11
3,907,"4,157","1,279",252,441,210,216,31,129,"2,878","1,435","1,009",268,112,10,44
4,90901,"4,026",0,0,0,0,0,0,0,"4,026","2,439",904,479,204,0,0
5,91001,"2,479","1,217",300,268,206,205,113,125,"1,262",466,555,175,0,17,49
6,911,"4,344","2,097",368,453,695,333,51,197,"2,247","1,251",554,342,63,0,37
7,912,"2,714","1,189",78,69,534,302,98,108,"1,525",768,457,259,14,0,27
8,913,"1,741",271,46,17,42,151,15,0,"1,470",669,523,193,0,22,63
9,914,"2,008",716,176,75,202,193,0,70,"1,292",738,124,280,0,0,150


In [ ]:
Owner occupied:	Moved in 2005 or later
df = df.drop(columns=columns_to_drop, axis=1)

,Total population in occupied housing units:,Owner occupied:,Moved in 2005 or later,Moved in 2000 to 2004,Moved in 1990 to 1999,Moved in 1980 to 1989,Moved in 1970 to 1979,Moved in 1969 or earlier,Renter occupied:,Moved in 2005 or later,Moved in 2000 to 2004,Moved in 1990 to 1999,Moved in 1980 to 1989,Moved in 1970 to 1979,Moved in 1969 or earlier
1,"4,658","1,351",341,311,441,56,39,163,"3,307","1,678",713,530,180,132,74
2,±561,±442,±324,±240,±257,±74,±60,±82,±515,±474,±322,±234,±144,±105,±66
3,"2,075",214,80,20,9,0,0,105,"1,861",810,641,318,29,0,63
4,±340,±175,±103,±31,±14,±127,±127,±142,±358,±331,±510,±297,±53,±127,±63
5,"2,525",591,151,34,167,56,90,93,"1,934","1,142",557,204,0,20,11
6,±409,±302,±158,±52,±180,±67,±90,±83,±399,±328,±361,±166,±127,±33,±18
7,"4,157","1,279",252,441,210,216,31,129,"2,878","1,435","1,009",268,112,10,44
8,±483,±391,±132,±281,±136,±156,±34,±102,±469,±457,±359,±141,±76,±17,±51
9,"4,026",0,0,0,0,0,0,0,"4,026","2,439",904,479,204,0,0
10,±413,±127,±127,±127,±127,±127,±127,±127,±413,±477,±310,±285,±156,±127,±127


In [ ]:
owner = "Owner_occupied_"
owner_subyear= [4,5,6,7,8,9]

# Add the prefix to the selected columns for each row
for col in owner_subyear:
    df[col] = owner + df[col].astype(str)

In [ ]:
df

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Unnamed: 0,Label,Total population in occupied housing units:,Owner occupied:,Moved in 2005 or later,Owner_occupied_Moved in 2000 to 2004,Owner_occupied_Moved in 1990 to 1999,Owner_occupied_Moved in 1980 to 1989,Owner_occupied_Moved in 1970 to 1979,Owner_occupied_Moved in 1969 or earlier,Owner_occupied_Renter occupied:,Moved in 2005 or later,Moved in 2000 to 2004,Moved in 1990 to 1999,Moved in 1980 to 1989,Moved in 1970 to 1979,Moved in 1969 or earlier
1,"Census Tract 901, Suffolk County, Massachusetts",Estimate,"4,658","1,351",341,Owner_occupied_311,Owner_occupied_441,Owner_occupied_56,Owner_occupied_39,Owner_occupied_163,"Owner_occupied_3,307","1,678",713,530,180,132,74
2,Unnamed: 2,Margin of Error,±561,±442,±324,Owner_occupied_±240,Owner_occupied_±257,Owner_occupied_±74,Owner_occupied_±60,Owner_occupied_±82,Owner_occupied_±515,±474,±322,±234,±144,±105,±66
3,"Census Tract 902, Suffolk County, Massachusetts",Estimate,"2,075",214,80,Owner_occupied_20,Owner_occupied_9,Owner_occupied_0,Owner_occupied_0,Owner_occupied_105,"Owner_occupied_1,861",810,641,318,29,0,63
4,Unnamed: 4,Margin of Error,±340,±175,±103,Owner_occupied_±31,Owner_occupied_±14,Owner_occupied_±127,Owner_occupied_±127,Owner_occupied_±142,Owner_occupied_±358,±331,±510,±297,±53,±127,±63
5,"Census Tract 903, Suffolk County, Massachusetts",Estimate,"2,525",591,151,Owner_occupied_34,Owner_occupied_167,Owner_occupied_56,Owner_occupied_90,Owner_occupied_93,"Owner_occupied_1,934","1,142",557,204,0,20,11
6,Unnamed: 6,Margin of Error,±409,±302,±158,Owner_occupied_±52,Owner_occupied_±180,Owner_occupied_±67,Owner_occupied_±90,Owner_occupied_±83,Owner_occupied_±399,±328,±361,±166,±127,±33,±18
7,"Census Tract 907, Suffolk County, Massachusetts",Estimate,"4,157","1,279",252,Owner_occupied_441,Owner_occupied_210,Owner_occupied_216,Owner_occupied_31,Owner_occupied_129,"Owner_occupied_2,878","1,435","1,009",268,112,10,44
8,Unnamed: 8,Margin of Error,±483,±391,±132,Owner_occupied_±281,Owner_occupied_±136,Owner_occupied_±156,Owner_occupied_±34,Owner_occupied_±102,Owner_occupied_±469,±457,±359,±141,±76,±17,±51
9,"Census Tract 909.01, Suffolk County, Massachus...",Estimate,"4,026",0,0,Owner_occupied_0,Owner_occupied_0,Owner_occupied_0,Owner_occupied_0,Owner_occupied_0,"Owner_occupied_4,026","2,439",904,479,204,0,0


In [ ]:
from re import I
def process_excel_file(year, excel_path, list_index):
    df = pd.read_excel(excel_path, sheet_name=1, header=None)

    header = df.iloc[0:3].apply(lambda x: '_'.join(x.astype(str)), axis=0)
    df.columns = header
    df = df[3:]

    df = df.set_index(df.columns[0])
    df = df.transpose()

    df = df[df.reset_index().index % 2 == 0]

    # Calculate the starting and ending row indices based on the list_index
    start_row = list_index * 4
    end_row = (list_index + 1) * 4

    new_df = df.iloc[start_row:end_row].copy()
    new_df.insert(0, 'year', year)
    new_df = new_df.reset_index()

    if isinstance(new_df.iat[0, 0], str) and len(new_df.iat[0, 0]) > 23:
        new_df.iat[0, 0] = new_df.iat[0, 0][-23:]

    new_df = new_df.rename(columns={'index': 'estimate_type'})
    new_df['estimate_type'] = new_df['estimate_type'].str[4:]
    new_df['Total'] = pd.to_numeric(new_df['Total'].str.replace(',', ''), errors='coerce').astype(float)

    return new_df